Ở đây đã đưa ra các documents hay texts đã được xử lý rồi. Bình thường nếu ở dạng thô nó sẽ là các đoạn văn chẳng hạn.

In [ ]:
# Ví dụ có raw text
#  doc1 = "anh yêu em nhiều lắm"
# doc2 = "em không yêu anh"
# doc3 = "chúng ta không yêu nhau"

# doc_1 = doc1.split()
# doc_2 = doc2.split()
# doc_3 = doc3.split()


In [8]:
doc_1 = ["red", "green", "blue", "yellow", "red", "red"]
doc_2 = ["red", "pink", "white", "dark", "orange", "pink"]
doc_3 = ["green", "yellow", "white", "white", "purpil"]

Đi tạo corpus

In [9]:
corpus = set(doc_1 + doc_2 + doc_3)
corpus

{'blue', 'dark', 'green', 'orange', 'pink', 'purpil', 'red', 'white', 'yellow'}

Đi tạo dictionay cho mỗi document với keys là các words, values là số lần xuất hiện của word trong document đó

In [10]:
# https://www.w3schools.com/python/ref_dictionary_fromkeys.asp
dict_doc_1 = dict.fromkeys(corpus, 0)
dict_doc_2 = dict.fromkeys(corpus, 0)
dict_doc_3 = dict.fromkeys(corpus, 0)


In [11]:
for word in doc_1:
    dict_doc_1[word] += 1

for word in doc_2:
    dict_doc_2[word] += 1

for word in doc_3:
    dict_doc_3[word] += 1

dict_doc_1

{'dark': 0,
 'blue': 1,
 'white': 0,
 'orange': 0,
 'green': 1,
 'yellow': 1,
 'red': 3,
 'pink': 0,
 'purpil': 0}

Đi tạo DataFrame từ các dictionaries tương ứng với các documents. Chính xác là từ **list of dictionaries**. Khi đó các keys chính là các columns của dataframe. https://stackoverflow.com/questions/20638006/convert-list-of-dictionaries-to-a-pandas-dataframe

In [12]:
import pandas as pd

df = pd.DataFrame([dict_doc_1, dict_doc_2, dict_doc_3])
df.head()

,dark,blue,white,orange,green,yellow,red,pink,purpil
0,0,1,0,0,1,1,3,0,0
1,1,0,1,1,0,0,1,2,0
2,0,0,2,0,1,1,0,0,1


## Tính TF

In [13]:
def TF(dict_doc, doc):
    """ 
        Ở đây sẽ tính TF cho tất cả các từ trong trong document 
        dict_doc: dictionary cho document
        doc: document - chính là list các từ trong document (có thể lặp)
    """
    tf_dict = {}
    # độ dài một đoạn text (document)
    num_words = len(doc)
    for word, count in dict_doc.items():
        tf_dict[word] = count / num_words
    return tf_dict



In [16]:
tf_doc_1 = TF(dict_doc_1, doc_1)
tf_doc_2 = TF(dict_doc_2, doc_2)
tf_doc_3 = TF(dict_doc_3, doc_3)

tf_doc_1


{'dark': 0.0,
 'blue': 0.16666666666666666,
 'white': 0.0,
 'orange': 0.0,
 'green': 0.16666666666666666,
 'yellow': 0.16666666666666666,
 'red': 0.5,
 'pink': 0.0,
 'purpil': 0.0}

**Chú ý**: Ở đây tf_doc cho mỗi document đều chứa tất cả các keys là các từ trong corpus. Nếu value của key nào bằng 0 chứng tỏ trong document đó không chứa key hay word đó.

In [17]:
# `pink` xuất hiện hai lần trong `doc_2` nên có TF lớn hơn gấp đôi các từ còn lại.
tf_doc_2

{'dark': 0.16666666666666666,
 'blue': 0.0,
 'white': 0.16666666666666666,
 'orange': 0.16666666666666666,
 'green': 0.0,
 'yellow': 0.0,
 'red': 0.16666666666666666,
 'pink': 0.3333333333333333,
 'purpil': 0.0}

## Tính IDF

$$idf(t, D) = \log\frac{|D|}{|{d \in D: t \in d}|}$$
trong đó: 
- $|D|$ - tổng số văn bản trong corpus D (gồm nhiều văn bản)
- $|{d \in D: t \in d}|$ - số văn bản chứa từ $t$ trong corpus D

In [20]:
import math 

def IDF(corpus):
    """
        corpus: corpus hay từ điển, ở đây là list các từ duy nhất
        được lấy từ các documents
        corpus chính là list of dictionaries cho mỗi document
        Chú ý các keys của mỗi dictionary là như nhau và chính là các từ trong corpus
        Đây là đặc điểm do chúng ta làm như vậy
    """
    # kích thước corpus - tương ứng với |D| trong công thức
    D = len(corpus)

    # tạo dictionay với các keys là các từ trong corpus, values mặc định ban đầu bằng 0
    dict_idf = dict.fromkeys(corpus[0].keys(), 0)

    for doc in corpus:  # chúng ta truyền corpus là list of dictionaries của các doc
        for word, val in doc.items():
            # nếu word có trong doc đang xét sẽ tăng value của dict_idf lên 1, thể hiện nó xuất hiện trong doc
            if val > 0:
                dict_idf[word] += 1

    for word, val in dict_idf.items():
        dict_idf[word] = math.log(D / val)  # val ở đây thực chất là dict_idf[word]

    # trả lại dict_idf - idf cho các từ trong corpus
    return dict_idf
    


In [22]:
dict_idf = IDF([dict_doc_1, dict_doc_2, dict_doc_3])
dict_idf

{'dark': 1.0986122886681098,
 'blue': 1.0986122886681098,
 'white': 0.4054651081081644,
 'orange': 1.0986122886681098,
 'green': 0.4054651081081644,
 'yellow': 0.4054651081081644,
 'red': 0.4054651081081644,
 'pink': 1.0986122886681098,
 'purpil': 1.0986122886681098}

## Tính TF-IDF

$$\text{TF-IDF}(t, d, D) = TF(t, d) \times IDF(t, D)$$

In [23]:
def TFIDF(tf, idf):
    """
        tf: tf_dict_doc chính là dictionary lưu tf cho các từ trong doc (ở đây chứa tất cả các từ trong corpus)
        idf: dict_idf chính là dictionary chứa idf của các từ trong corpus 
    """
    # tạo empty dictionary để lưu TF-IDF
    tfidf = {}

    for word, val in tf.items():
        tfidf[word] = val * idf[word]
    
    return tfidf
    

In [24]:
tfidf_doc_1 = TFIDF(tf_doc_1, dict_idf)
tfidf_doc_2 = TFIDF(tf_doc_2, dict_idf)
tfidf_doc_3 = TFIDF(tf_doc_3, dict_idf)



In [25]:
tfidf_doc_1

{'dark': 0.0,
 'blue': 0.1831020481113516,
 'white': 0.0,
 'orange': 0.0,
 'green': 0.06757751801802739,
 'yellow': 0.06757751801802739,
 'red': 0.2027325540540822,
 'pink': 0.0,
 'purpil': 0.0}

Gộp vào dataframe

In [26]:
tfidf = pd.DataFrame([tfidf_doc_1, tfidf_doc_2, tfidf_doc_3])
tfidf.head()

,dark,blue,white,orange,green,yellow,red,pink,purpil
0,0.000000,0.183102,0.000000,0.000000,0.067578,0.067578,0.202733,0.000000,0.000000
1,0.183102,0.000000,0.067578,0.183102,0.000000,0.000000,0.067578,0.366204,0.000000
2,0.000000,0.000000,0.162186,0.000000,0.081093,0.081093,0.000000,0.000000,0.219722


Thật vậy với `doc_1` và từ `red` nhận thấy $tf(red, doc_1) = \frac{3}{6} = 0.5$ và $idf(red, D) = \log(\frac{3}{2})$, do đó nhận được $tfidf(red, doc_1, corpus) = 0.202725$

**Chú ý**: cái mình nhận được ở trên chính là **tf-vectorization**, mỗi docmument bây giờ được biểu diễn dưới dạng feature vector (tương tự như Bag of word) nhwung bây giờ với mỗi từ ta thay giá trị tf-idf của từ đó.